# Cybersecurity advanced
## Lab 02: Firewall


Links to Syllabus

- [Theoretical](https://hogenttin.github.io/cybersecurity-advanced/lesson-2/theory/)
- [Practical](https://hogenttin.github.io/cybersecurity-advanced/lesson-2/lab/)

### Insecure "Fake Internet"

visiting http://www.cybersec.internal

![internal](./DocImg/lab1-3.png)

visiting and using insecure application

![insec app](./DocImg/Lab2-1.png)

SSH connection using vagrant login → possible

![ssh](./DocImg/Lab2-2.png)

### Firewall

For the firewall nftables is used. The following minimal firewall settings are used:

```bash
#!/usr/sbin/nft -f
flush ruleset
table ip filter {
    chain forward {
        type filter hook forward priority filter; policy drop;
        ip saddr 192.168.62.254 ip daddr 0.0.0.0/0 tcp dport 22 accept
        ip saddr 0.0.0.0/0 ip daddr 0.0.0.0/0 icmp type echo-request accept
        ip saddr 0.0.0.0/0 ip daddr 172.30.0.4 udp dport 53 accept
        ip saddr 0.0.0.0/0 ip daddr 172.30.0.10 tcp dport { 80, 443 } accept
        ip saddr 172.30.0.10 ip daddr 172.30.64.15 accept
        ip saddr 172.30.64.15 ip daddr 172.30.0.10 accept
        ip saddr { 172.30.0.0/16 } ip daddr 0.0.0.0/0 accept
        ct state established,related accept
    }
}
```

***Note: It is important to make sure that the formatting settings in LF and not in CTRF***

OR

IPTables:
```bash
# accept SSH-traffic to 192.168.62.253 on port 22 via eth1:
sudo iptables -A INPUT -p tcp --dport 22 -d 192.168.62.253 -i eth1 -j ACCEPT

# accpet traffic to 192.168.62.253 on port 2222 via eth0:
sudo iptables -A INPUT -p tcp --dport 2222 -d 192.168.62.253 -i eth1 -j ACCEPT

# accpet MySQL-traffic to 192.168.62.253 on port 3306 via eth1:
sudo iptables -A INPUT -p tcp --dport 3306 -d 192.168.62.253 -i eth1 -j ACCEPT

# accept all incomming traffic to 172.30.255.254 via eth2:
sudo iptables -A INPUT -d 172.30.255.254 -i eth2 -j ACCEPT

# Accept MySQL-traffic op port 3306:
sudo iptables -A INPUT -p tcp --dport 3306 -j ACCEPT

# accept incomming traffic, part of or related to connections to 192.168.62.253 via eth1:
sudo iptables -I INPUT -i eth1 -d 192.168.62.253 -m state --state ESTABLISHED,RELATED -j ACCEPT

# Set the default protocol for incomming traffic on DROP:
sudo iptables -P INPUT DROP

# Set the default protocol for passing traffic on DROP:
sudo iptables -P FORWARD DROP

# Set the default protocol for outgoing traffic on ACCEPT:
sudo iptables -P OUTPUT ACCEPT
```
To save rules: iptables-save > /etc/iptables.rules


To revert: sudo iptables-restore < /etc/iptables/rules.v4


### Ports

After the `nmap -p 80,22,666 172.30.0.10`

The result is the following:

- 80 → open (ssh)
- 22 → open (http)
- 666 → closed (doom)

![nmap result](./DocImg/Lab2-3.png)

sudo dnf install -y autoconf automake diffutils file-devel gcc gcc-c++ git jansson-devel jq libcap-ng-devel libevent-devel libmaxminddb-devel libnet-devel libnetfilter_queue-devel libnfnetlink-devel libpcap-devel libtool libyaml-devel lua-devel lz4-devel make nss-devel pcre2-devel pkgconfig python3-devel python3-sphinx python3-yaml sudo which zlib-devel